Google Colab stuff

In [36]:
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
%pip install transformers
%pip install pytorch
%pip install scikit-learn
%pip install conllu
%pip install tqdm

import sys
sys.path.append("/content/drive/My Drive/nlp2-probing-lms/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, 

In [1]:
#
## Your code for initializing the transformer model(s)
#
# Note that transformer models use their own `tokenizer`, that should be loaded in as well.
#
from transformers import *
import transformers
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

c:\Users\Cyproxius\anaconda3\envs\nlp2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Cyproxius\anaconda3\envs\nlp2\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
loading file vocab.json from cache at C:\Users\Cyproxius/.cache\huggingface\hub\models--distilgpt2\snapshots\38cc92ec43315abd5136313225e95acc5986876c\vocab.json
loading file merges.txt from cache at C:\Users\Cyproxius/.cache\huggingface\hub\models--

In [5]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/file/d/19Lp3AM4NEPycp_IBgoHfLc_V456pmUom/view?usp=sharing
# You can read more about this model in the original paper here: https://arxiv.org/pdf/1803.11138.pdf
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
from collections import defaultdict
from lstm.model import RNNModel
import torch

# pre_path = "/content/drive/My Drive/nlp2-probing-lms/"
pre_path = ""

model_location = f'{pre_path}state_dictLSTM.pt'  # <- point this to the location of the Gulordava .pt file
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open(f'{pre_path}lstm/vocab.txt', encoding="utf8") as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)

In [6]:
# READ DATA
from typing import List
from conllu import parse_incr, TokenList


# If stuff like `: str` and `-> ..` seems scary, fear not! 
# These are type hints that help you to understand what kind of argument and output is expected.
def parse_corpus(filename: str) -> List[TokenList]:
    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses


ud_parses = parse_corpus(f'{pre_path}data/sample/en_ewt-ud-train.conllu')

In [7]:
# FETCH SENTENCE REPRESENTATIONS
from torch import Tensor
import pickle
from tqdm import tqdm

# Should return a tensor of shape (num_tokens_in_corpus, representation_size)
# Make sure you correctly average the subword representations that belong to 1 token!

def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer, concat=True, layer=-1) -> Tensor:
    
    model.eval()
    # Initialize an empty list to store sentence representations
    corpus_rep = []
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Transformer model
    if isinstance(tokenizer, transformers.PreTrainedTokenizer):
        # Loop through the parses
        for parse in tqdm(ud_parses):
            prev_token = None
            encoded_full = []
            encoded = []
            tokens = []
            # Loop through the tokens in the parse
            for token in parse:
                # Check if the previous token has a misc attribute and SpaceAfter key
                if prev_token is None or ('misc' in prev_token and prev_token['misc'] is not None and 'SpaceAfter' in prev_token['misc'] and prev_token['misc']['SpaceAfter'] == 'No'):
                    tokens.append(token['form'])
                else:
                    tokens.append(' ' + token['form'])
                prev_token = token

            # Encode each word using the tokenizer
            for token in tokens:
                with torch.no_grad():
                    encoded_token = tokenizer.encode(token, add_special_tokens=False)
                    encoded.extend(encoded_token)
                    encoded_full.append(encoded_token)

            # Convert the encoded subword to a tensor        
            input_ids = torch.tensor(encoded).unsqueeze(0)
            input_ids = input_ids.to(device)
            # Get the sentence representation from the model's output
            with torch.no_grad():
                outputs = model(input_ids, output_hidden_states=True)

            last_layer_hidden_state = outputs.hidden_states[layer]
            sen_rep = last_layer_hidden_state.squeeze()
            # Fix for one token sentences
            if len(parse) <= 1:
                sen_rep = sen_rep.unsqueeze(0)
            # Average the representations of subwords
            sen_rep_avg = []
            i = 0
            for word in encoded_full:
                j = len(word)
                sen_rep_avg.append(torch.mean(sen_rep[i:i+j], dim=0, keepdim=True))
                i += j
            tensor = torch.stack(sen_rep_avg, dim=0).squeeze(1)
            if tensor.shape[0] == 1:
                tensor = tensor.view(1, -1, tensor.shape[-1])[:, 0, :]      
            corpus_rep.append(tensor)
        
    # LSTM model
    else:
        for parse in ud_parses:
            encoded = [tokenizer.get(token['form'].replace(' ', ''), tokenizer['<unk>']) for token in parse]
            input_ids = torch.tensor(encoded).unsqueeze(0)
            input_ids = input_ids.to(device)
            hidden = model.init_hidden(1)
            with torch.no_grad():
                tensor = lstm(input_ids, hidden).squeeze()
            if len(tensor.shape) == 1:
                tensor = tensor.unsqueeze(0)
            corpus_rep.append(tensor)
    if concat:
        corpus_rep = torch.cat(corpus_rep, dim=0)
    
    return corpus_rep

In [9]:
def error_msg(model_name, gold_embs, embs, i2w):
    with open(f'{pre_path}{model_name}_tokens1.pickle', 'rb') as f:
        sen_tokens = pickle.load(f)
        
    diff = torch.abs(embs - gold_embs)
    max_diff = torch.max(diff)
    avg_diff = torch.mean(diff)
    
    print(f"{model_name} embeddings don't match!")
    print(f"Max diff.: {max_diff:.4f}\nMean diff. {avg_diff:.4f}")

    print("\nCheck if your tokenization matches with the original tokenization:")
    for idx in sen_tokens.squeeze():
        if isinstance(i2w, list):
            token = i2w[idx]
        else:
            token = i2w.convert_ids_to_tokens(idx.item())
        print(f"{idx:<6} {token}")


def assert_sen_reps(model, tokenizer, lstm, vocab):
    with open(f'{pre_path}distilgpt2_emb1.pickle', 'rb') as f:
        distilgpt2_emb1 = pickle.load(f)
        
    with open(f'{pre_path}lstm_emb1.pickle', 'rb') as f:
        lstm_emb1 = pickle.load(f)
    
    corpus = parse_corpus(f'{pre_path}data/sample/en_ewt-ud-train.conllu')[:1]
    
    own_distilgpt2_emb1 = fetch_sen_reps(corpus, model, tokenizer)
    own_lstm_emb1 = fetch_sen_reps(corpus, lstm, vocab)
    
    # assert distilgpt2_emb1.shape == own_distilgpt2_emb1.shape, \
    #     f"Distilgpt2 shape mismatch: {distilgpt2_emb1.shape} (gold) vs. {own_distilgpt2_emb1.shape} (yours)"
    # assert lstm_emb1.shape == own_lstm_emb1.shape, \
    #     f"LSTM shape mismatch: {lstm_emb1.shape} (gold) vs. {own_lstm_emb1.shape} (yours)"

    # if not torch.allclose(distilgpt2_emb1, own_distilgpt2_emb1, rtol=1e-3, atol=1e-3):
    #     error_msg("distilgpt2", distilgpt2_emb1, own_distilgpt2_emb1, tokenizer)
    # if not torch.allclose(lstm_emb1, own_lstm_emb1, rtol=1e-3, atol=1e-3):
    #     error_msg("lstm", lstm_emb1, own_lstm_emb1, list(vocab.keys()))


assert_sen_reps(model, tokenizer, lstm, vocab)

100%|██████████| 1/1 [00:01<00:00,  1.38s/it]


In [10]:
# FETCH POS LABELS
from typing import Optional, Dict, Tuple

# Should return a tensor of shape (num_tokens_in_corpus,)
# Make sure that when fetching these pos tags for your train/dev/test corpora you share the label vocabulary.
def fetch_pos_tags(ud_parses: List[TokenList], pos_vocab: Optional[Dict[str, int]] = None) -> Tuple[Tensor, Dict[str, int]]:
    
    # Create label dictionary if not passed to function
    if pos_vocab is None:
        pos_vocab = {}
        pos_counter = 0
    else:
        pos_counter = len(pos_vocab.values())

    # Keep a list of all POS tag labels
    all_tags = []

    for parse in ud_parses:
        for token in parse:
            # Extract the POS tag for the given token
            pos_tag = token['upostag']
            
            # If the token is not yet in the vocabulary, add it
            if pos_tag not in pos_vocab.keys():
                pos_vocab[pos_tag] = pos_counter
                pos_counter += 1
            
            # Add the label encoding of the POS tag
            all_tags.append(pos_vocab[pos_tag])

    return torch.tensor(all_tags), pos_vocab


fetch_pos_tags(ud_parses)

(tensor([0, 1, 0,  ..., 3, 0, 4]),
 {'PROPN': 0,
  'PUNCT': 1,
  'ADJ': 2,
  'NOUN': 3,
  'VERB': 4,
  'DET': 5,
  'ADP': 6,
  'AUX': 7,
  'PRON': 8,
  'PART': 9,
  'SCONJ': 10,
  'NUM': 11,
  'ADV': 12,
  'CCONJ': 13})

In [48]:
import os

# Function that combines the previous functions, and creates 2 tensors for a .conllu file: 
# 1 containing the token representations, and 1 containing the (tokenized) pos_tags.

def create_data(ud_parses, filename: str, lm, w2i, pos_vocab=None, layer=-1):
    new_parses = parse_corpus(filename)
    
    sen_reps = fetch_sen_reps(new_parses, lm, w2i, concat=False, layer=layer)
    pos_tags, pos_vocab = fetch_pos_tags(new_parses, pos_vocab=pos_vocab)
    
    return sen_reps, pos_tags, pos_vocab

# Trees

In [12]:
# In case you want to transform your conllu tree to an nltk.Tree, for better visualisation

def rec_tokentree_to_nltk(tokentree):
    token = tokentree.token["form"]
    tree_str = f"({token} {' '.join(rec_tokentree_to_nltk(t) for t in tokentree.children)})"

    return tree_str


def tokentree_to_nltk(tokentree):
    from nltk import Tree as NLTKTree

    tree_str = rec_tokentree_to_nltk(tokentree)

    return NLTKTree.fromstring(tree_str)

In [13]:
%pip install ete3
from ete3 import Tree as EteTree


class FancyTree(EteTree):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, format=1, **kwargs)
        
    def __str__(self):
        return self.get_ascii(show_internal=True)
    
    def __repr__(self):
        return str(self)


def rec_tokentree_to_ete(tokentree):
    idx = str(tokentree.token["id"])
    children = tokentree.children
    if children:
        return f"({','.join(rec_tokentree_to_ete(t) for t in children)}){idx}"
    else:
        return idx
    
def tokentree_to_ete(tokentree):
    newick_str = rec_tokentree_to_ete(tokentree)

    return FancyTree(f"{newick_str};")

  Using cached ete3-3.1.3.tar.gz (4.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ete3: filename=ete3-3.1.3-py3-none-any.whl size=2273809 sha256=f4e40aa3763955f96acf2d8c6e4fb3fbdd914f877f750a736b8ac39c3be87c36
  Stored in directory: c:\users\cyproxius\appdata\local\pip\cache\wheels\a0\72\00\1982bd848e52b03079dbf800900120bc1c20e92e9a1216e525
Successfully built ete3
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Let's check if it works!
# We can read in a corpus using the code that was already provided, and convert it to an ete3 Tree.

def parse_corpus(filename):
    from conllu import parse_incr

    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

corpus = parse_corpus(f'{pre_path}data/sample/en_ewt-ud-train.conllu')
item = corpus[0]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree)


   /-2
  |
  |--3
  |
  |--4
  |
  |   /6 /-5
  |  |
  |  |   /-9
  |  |  |
  |  |  |--10
  |  |  |
  |  |  |--11
  |  |-8|
  |  |  |--12
  |-7|  |
  |  |  |--13
  |  |  |
  |  |   \15/-14
-1|  |
  |  |   /-16
  |  |  |
  |  |  |--17
  |  |  |
  |   \18   /-19
  |     |  |
  |     |  |--20
  |     |  |
  |     |  |-23/-22
  |      \21
  |        |--24
  |        |
  |        |   /-25
  |        |  |
  |         \28--26
  |           |
  |            \-27
  |
   \-29


In [15]:
def create_gold_distances(corpus):
    all_distances = []

    for item in corpus:
        tokentree = item.to_tree()
        ete_tree = tokentree_to_ete(tokentree)

        sen_len = len(ete_tree.search_nodes())
        distances = torch.zeros((sen_len, sen_len))

        # Compute all the distances
        for i in range(sen_len):
            for j in range(sen_len):
                if i == j:
                    distances[i,j] = 0.0
                    distances[j,i] = 0.0
                else:
                    distances[i,j] = ete_tree.get_distance(str(i+1), str(j+1))
                    distances[j,i] = ete_tree.get_distance(str(i+1), str(j+1))

        all_distances.append(distances)

    return all_distances

In [16]:
from scipy.sparse.csgraph import minimum_spanning_tree
import torch


def create_mst(distances):
    distances = torch.triu(distances).detach().numpy()
    mst = minimum_spanning_tree(distances).toarray()
    mst[mst>0] = 1.
    
    return mst

In [17]:
item = corpus[5]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree, '\n')

gold_distance = create_gold_distances(corpus[5:6])[0]
print(gold_distance, '\n')

mst = create_mst(gold_distance)
print(mst)
print(gold_distance, '\n')


   /2 /-1
  |
  |--3
  |
  |--4
  |
  |   /-6
  |  |
-5|  |--7
  |-8|
  |  |   /-9
  |  |  |
  |   \12--10
  |     |
  |      \-11
  |
   \-13 

tensor([[0., 1., 3., 3., 2., 4., 4., 3., 5., 5., 5., 4., 3.],
        [1., 0., 2., 2., 1., 3., 3., 2., 4., 4., 4., 3., 2.],
        [3., 2., 0., 2., 1., 3., 3., 2., 4., 4., 4., 3., 2.],
        [3., 2., 2., 0., 1., 3., 3., 2., 4., 4., 4., 3., 2.],
        [2., 1., 1., 1., 0., 2., 2., 1., 3., 3., 3., 2., 1.],
        [4., 3., 3., 3., 2., 0., 2., 1., 3., 3., 3., 2., 3.],
        [4., 3., 3., 3., 2., 2., 0., 1., 3., 3., 3., 2., 3.],
        [3., 2., 2., 2., 1., 1., 1., 0., 2., 2., 2., 1., 2.],
        [5., 4., 4., 4., 3., 3., 3., 2., 0., 2., 2., 1., 4.],
        [5., 4., 4., 4., 3., 3., 3., 2., 2., 0., 2., 1., 4.],
        [5., 4., 4., 4., 3., 3., 3., 2., 2., 2., 0., 1., 4.],
        [4., 3., 3., 3., 2., 2., 2., 1., 1., 1., 1., 0., 3.],
        [3., 2., 2., 2., 1., 3., 3., 2., 4., 4., 4., 3., 0.]]) 

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0

In [18]:
def edges(mst):
    # Your code for retrieving the edges from the MST matrix
    edges = set()
    for i, node1 in enumerate(mst):
        for j, node2 in enumerate(node1):
            if node2 == 1:
                edges.add((i,j))
                edges.add((j,i))

    return edges

def calc_uuas(pred_distances, gold_distances):
    uuas = None
    # Find the indices of non-padded rows and columns
    non_padded_rows = torch.any(gold_distances != -1, dim=1)
    non_padded_cols = torch.any(gold_distances != -1, dim=0)

    # Slice both tensors using the non-padded indices
    pred_distances = pred_distances[non_padded_rows][:, non_padded_cols]
    gold_distances = gold_distances[non_padded_rows][:, non_padded_cols]
    
    pred_mst = create_mst(pred_distances)
    gold_mst = create_mst(gold_distances)
    pred_edges = edges(pred_mst)
    gold_edges = edges(gold_mst)
    correct = pred_edges.intersection(gold_edges)
    if len(gold_edges) == 0:
        uuas = 1
    else:
        uuas = len(correct) / len(gold_edges)
    return uuas

# Structural probing

In [19]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        self.proj_norms = nn.Parameter(data=torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        nn.init.uniform_(self.proj_norms, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """
        transformed = torch.matmul(batch, self.proj)
        transformed_norms = torch.matmul(batch, self.proj_norms)
        
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)
        
        squared_norms = transformed_norms.pow(2).sum(-1)

        return squared_distances, squared_norms

    
    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents

In [71]:
from torch import optim
import math

'''
Similar to the `create_data` method of the previous notebook, I recommend you to use a method 
that initialises all the data of a corpus. Note that for your embeddings you can use the 
`fetch_sen_reps` method again. However, for the POS probe you concatenated all these representations into 
1 big tensor of shape (num_tokens_in_corpus, model_dim). 

The StructuralProbe expects its input to contain all the representations of 1 sentence, so I recommend you
to update your `fetch_sen_reps` method in a way that it is easy to retrieve all the representations that 
correspond to a single sentence.
''' 

def init_corpus(path, concat=False, cutoff=None, layer=-1):
    """ Initialises the data of a corpus.
    
    Parameters
    ----------
    path : str
        Path to corpus location
    concat : bool, optional
        Optional toggle to concatenate all the tensors
        returned by `fetch_sen_reps`.
    cutoff : int, optional
        Optional integer to "cutoff" the data in the corpus.
        This allows only a subset to be used, alleviating 
        memory usage.
    """
    corpus = parse_corpus(path)[:cutoff]
    embs = fetch_sen_reps(corpus, lstm, vocab, concat=concat, layer=layer)
#     embs = fetch_sen_reps(corpus, model, tokenizer, concat=concat, layer=layer)    
    gold_distances = create_gold_distances(corpus)
    
    return gold_distances, embs


# I recommend you to write a method that can evaluate the UUAS & loss score for the dev (& test) corpus.
# Feel free to alter the signature of this method.
def evaluate_probe(probe, _data):
    loss_function =  L1DistanceLoss()
    probe.eval()
    
    with torch.no_grad():
        distances = _data[0]
        embs = _data[1]
        
        # Make big tensor of shape (batch_size, max_seq_len, representation_dim)
        # from the representations in the batch using zero-padding
        max_seq_len = max([tensor.shape[0] for tensor in embs])
        batch_size = len(embs)
        representation_dim = embs[0].shape[1]
        embs_tensor = torch.zeros(batch_size, max_seq_len, representation_dim)
        for i, tensor in enumerate(embs):
            if tensor.shape[0] == 1:
                tensor = tensor.view(1, -1, representation_dim)[:, 0, :]
            seq_len = tensor.shape[0]
            embs_tensor[i, :seq_len] = tensor

        # Predict distances and norms using probe
        predicted_distances, predicted_norms = probe(embs_tensor)

        # Make big tensor of distances from batch padded with -1
        # resulting in true_distances of shape (batch_size, max_seq_len, max_seq_len)
        max_seq_len = max([tensor.shape[0] for tensor in distances])
        batch_size = len(distances)
        sentence_lengths = torch.zeros(batch_size, dtype=torch.long)
        true_distances = torch.full((batch_size, max_seq_len, max_seq_len), -1)
        for i, tensor in enumerate(distances):
            seq_len = tensor.shape[0]
            true_distances[i, :seq_len, :seq_len] = tensor
            sentence_lengths[i] = seq_len

        # Calculate loss and uass score
        loss, total_sents = loss_function(predicted_distances, true_distances, sentence_lengths)
        uuas_score = [calc_uuas(pred, true) for pred, true in zip(predicted_distances, true_distances)]
    
    return loss.item(), uuas_score


# Feel free to alter the signature of this method.
def train(_train_data, _dev_data, _test_data, device, epochs=1000):
    emb_dim = _train_data[1][0].shape[-1]
    rank = 64
    lr = 10e-4
    batch_size = 24
    dev_losses = []
    dev_uuases = []

    probe = StructuralProbe(emb_dim, rank, device)
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()

    for epoch in tqdm(range(epochs)):
        print("Epoch:", epoch)
        for i in range(0, len(_train_data[1]), batch_size):
            optimizer.zero_grad()

            # YOUR CODE FOR DOING A PROBE FORWARD PASS
            batch_embs = _train_data[1][i:i+batch_size]
            batch_distances = _train_data[0][i:i+batch_size]
            
            
            # Make big tensor of shape (batch_size, max_seq_len, representation_dim)
            # from the representations in the batch using zero-padding
            max_seq_len = max([tensor.shape[0] for tensor in batch_embs])
            embs_tensor = torch.zeros(len(batch_embs), max_seq_len, emb_dim)
            for j, tensor in enumerate(batch_embs):
                if tensor.shape[0] == 1:
                    tensor = tensor.view(1, -1, emb_dim)[:, 0, :]
                seq_len = tensor.shape[0]
                embs_tensor[j, :seq_len] = tensor
    
            # Predict distances and using probe
            predicted_distances, predicted_norms = probe(embs_tensor.to(device))
            
            # Make big tensor of shape (batch_size, max_seq_len, max_seq_len) from distances
            # in the batch using padding with -1
            max_seq_len = max([tensor.shape[0] for tensor in batch_distances])
            sentence_lengths = torch.zeros(len(batch_embs), dtype=torch.long)
            true_distances = torch.full((len(batch_embs), max_seq_len, max_seq_len), -1)
            for j, tensor in enumerate(batch_distances):
                seq_len = tensor.shape[0]
                true_distances[j, :seq_len, :seq_len] = tensor
                sentence_lengths[j] = seq_len

            # Calculate loss
            batch_loss, total_sents = loss_function(predicted_distances, true_distances, sentence_lengths)

            batch_loss.backward()
            optimizer.step()

        dev_loss, dev_uuas = evaluate_probe(probe, _dev_data)
        dev_losses.append(dev_loss)
        dev_uuases.append(sum(dev_uuas)/len(dev_uuas))
        

        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        scheduler.step(dev_loss)

    test_loss, test_uuas = evaluate_probe(probe, _test_data)
    return dev_losses, dev_uuases, test_loss, sum(test_uuas)/len(test_uuas)

# Amnesic probing

In [69]:
import numpy as np
import scipy

def get_probe_projection(probe):
    """
    Function to get the nullspace projection of a linear probe
    """

    null_space_proj = scipy.linalg.null_space(probe)
    return null_space_proj


def project_representations(data, probe):
    """
    Function to project all training data on the null space of the probe,
    as such effectively removing all information conveyed by the probe
    """
    # print(f"Probe shape: {probe.shape}")
    # print(f"Data shape: {data.shape}")

    null_space_proj = get_probe_projection(probe)
    # print(f"Null space projection shape: {null_space_proj.shape}")
    # print(f"Null space projection: {null_space_proj}")
    # print(f"Vector shape: {first_vec.shape}")
    # print(f"Projection shape: {proj.shape}")

    projected_data = [torch.from_numpy(np.array(elem)@null_space_proj) for elem in data]

    # print(f"full projection shape: {projected_data.shape}")
    # print(f"projection: {projected_data}")
    
    return projected_data

In [22]:
def get_struct_data(path):

    corpus = parse_corpus(path)
    gold_distances = create_gold_distances(corpus)
    
    return gold_distances

In [72]:
import time

num_layers = 7
epochs = 20
test_UUAS_per_layer = []
use_sample = True
model_name = "GPT"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# conllu_path = "/content/drive/My Drive/nlp2-probing-lms/data/"
conllu_path = "data/"

if use_sample:
  conllu_path += "sample/"

t = time.time()
print(f"Generating training distances...")
gold_distances_train = get_struct_data(f"{conllu_path}en_ewt-ud-train.conllu")
print(f"Done generating training distances. This took {time.time() - t}s")

t = time.time()
print(f"Generating validation distances...")
gold_distances_dev = get_struct_data(f"{conllu_path}en_ewt-ud-dev.conllu")
print(f"Done generating validation distances. This took {time.time() - t}s")

t = time.time()
print(f"Generating testing distances...")
gold_distances_test = get_struct_data(f"{conllu_path}en_ewt-ud-test.conllu")
print(f"Done generating testing distances. This took {time.time() - t}s")

for layer in range(num_layers):

    t = time.time()
    print(f"Generating training data...")
    GPT_train_x, GPT_train_y, GPT_train_vocab = create_data(
        ud_parses,
        os.path.join(f'{pre_path}data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
        model, 
        tokenizer,
        layer=layer
    )
    print(f"Done generating training data. This took {time.time() - t}s")

    t = time.time()
    print(f"Generating validation data...")
    GPT_dev_x, GPT_dev_y, _ = create_data(
        ud_parses,
        os.path.join(f'{pre_path}data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
        model, 
        tokenizer,
        pos_vocab=GPT_train_vocab,
        layer=layer
    )
    print(f"Done generating validation data. This took {time.time() - t}s")

    t = time.time()
    print(f"Generating test data...")
    GPT_test_x, GPT_test_y, _ = create_data(
        ud_parses,
        os.path.join(f'{pre_path}data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
        model,
        tokenizer,
        pos_vocab=GPT_train_vocab,
        layer=layer
    )
    print(f"Done generating test data. This took {time.time() - t}s")


    POS_probe = LogisticRegression()
    t = time.time()
    print(f"Fitting probe...")
    POS_probe.fit(torch.cat(GPT_train_x, dim=0), GPT_train_y)
    print(f"Done fitting POS probe. This took {time.time() - t}s")

    probe_weights = POS_probe.coef_

    # Get the "amnesic vectors", the representations with POS information removed
    GPT_train_projected = project_representations(GPT_train_x, probe_weights)
    GPT_dev_projected = project_representations(GPT_dev_x, probe_weights)
    GPT_test_projected = project_representations(GPT_test_x, probe_weights)

    t = time.time()
    print("Training structural probe...")
    # Train the structural probe on these amnesic vectors
    _, _, proj_GPT_test_loss, proj_GPT_test_uuas = train((gold_distances_train, GPT_train_projected),
                                                         (gold_distances_dev, GPT_dev_projected),
                                                         (gold_distances_test, GPT_test_projected),
                                                         device,
                                                         epochs)
    print(f"Done training structural probe. This took {time.time() - t}s")

    test_UUAS_per_layer.append(proj_GPT_test_uuas)

Generating training distances...
Done generating training distances. This took 6.644079208374023s
Generating validation distances...
Done generating validation distances. This took 2.4767580032348633s
Generating testing distances...
Done generating testing distances. This took 1.8606925010681152s
Generating training data...


100%|██████████| 90/90 [01:43<00:00,  1.16s/it]


Done generating training data. This took 104.05411553382874s
Generating validation data...


100%|██████████| 50/50 [00:52<00:00,  1.05s/it]


Done generating validation data. This took 52.390767097473145s
Generating test data...


...

Epoch: 19


100%|██████████| 20/20 [00:11<00:00,  1.77it/s]


Done training structural probe. This took 11.430310010910034s
Generating training data...


 71%|███████   | 64/90 [01:16<00:30,  1.19s/it]


KeyboardInterrupt: 

**Debug cells**

In [49]:
# import time

# num_layers = 7
# epochs = 20
# test_UUAS_per_layer = []
# use_sample = True
# model_name = "GPT"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # conllu_path = "/content/drive/My Drive/nlp2-probing-lms/data/"
# conllu_path = "data/"

# if use_sample:
#   conllu_path += "sample/"

# t = time.time()
# print(f"Generating training distances...")
# gold_distances_train = get_struct_data(f"{conllu_path}en_ewt-ud-train.conllu")
# print(f"Done generating training distances. This took {time.time() - t}s")

# t = time.time()
# print(f"Generating validation distances...")
# gold_distances_dev = get_struct_data(f"{conllu_path}en_ewt-ud-dev.conllu")
# print(f"Done generating validation distances. This took {time.time() - t}s")

# t = time.time()
# print(f"Generating testing distances...")
# gold_distances_test = get_struct_data(f"{conllu_path}en_ewt-ud-test.conllu")
# print(f"Done generating testing distances. This took {time.time() - t}s")

# layer = 1

# t = time.time()
# print(f"Generating training data...")
# GPT_train_x, GPT_train_y, GPT_train_vocab = create_data(
#     ud_parses,
#     os.path.join(f'{pre_path}data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
#     model, 
#     tokenizer,
#     layer=layer
# )
# print(f"Done generating training data. This took {time.time() - t}s")

# t = time.time()
# print(f"Generating validation data...")
# GPT_dev_x, GPT_dev_y, _ = create_data(
#     ud_parses,
#     os.path.join(f'{pre_path}data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
#     model, 
#     tokenizer,
#     pos_vocab=GPT_train_vocab,
#     layer=layer
# )
# print(f"Done generating validation data. This took {time.time() - t}s")

# t = time.time()
# print(f"Generating test data...")
# GPT_test_x, GPT_test_y, _ = create_data(
#     ud_parses,
#     os.path.join(f'{pre_path}data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
#     model,
#     tokenizer,
#     pos_vocab=GPT_train_vocab,
#     layer=layer
# )
# print(f"Done generating test data. This took {time.time() - t}s")


Generating training distances...
Done generating training distances. This took 6.569975137710571s
Generating validation distances...
Done generating validation distances. This took 2.455244779586792s
Generating testing distances...
Done generating testing distances. This took 1.8536860942840576s
Generating training data...


100%|██████████| 90/90 [01:41<00:00,  1.13s/it]


Done generating training data. This took 101.76648831367493s
Generating validation data...


100%|██████████| 50/50 [00:50<00:00,  1.02s/it]


Done generating validation data. This took 50.83025908470154s
Generating test data...


100%|██████████| 50/50 [00:50<00:00,  1.02s/it]

Done generating test data. This took 51.01414513587952s


In [55]:
# POS_probe = LogisticRegression()
# t = time.time()
# print(f"Fitting probe...")

# # print(len(GPT_train_x[2][0]))

# # print(torch.cat(GPT_train_x, dim=0).size())
# # print(torch.tensor(GPT_train_x).size())

# POS_probe.fit(torch.cat(GPT_train_x, dim=0), GPT_train_y)
# print(f"Done fitting POS probe. This took {time.time() - t}s")

# # Predict using the trained model
# # GPT_train_predictions = POS_probe.predict(GPT_train_x)
# # GPT_dev_predictions = POS_probe.predict(GPT_dev_x)
# # GPT_test_predictions = POS_probe.predict(GPT_test_x)

# # # Calculate accuracy scores
# # GPT_train_accuracy = accuracy_score(GPT_train_y, GPT_train_predictions)
# # GPT_dev_accuracy = accuracy_score(GPT_dev_y, GPT_dev_predictions)
# # GPT_test_accuracy = accuracy_score(GPT_test_y, GPT_test_predictions)

Fitting probe...
768
torch.Size([2301, 768])
Done fitting POS probe. This took 0.5420012474060059s


c:\Users\Cyproxius\anaconda3\envs\nlp2\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [70]:
# # print(GPT_train_x.shape())
# # print(GPT_train_projected.shape())

# probe_weights = POS_probe.coef_

# # Get the "amnesic vectors", the representations with POS information removed
# GPT_train_projected = project_representations(GPT_train_x, probe_weights)
# GPT_dev_projected = project_representations(GPT_dev_x, probe_weights)
# GPT_test_projected = project_representations(GPT_test_x, probe_weights)

# t = time.time()
# print("Training structural probe...")

# # Train the structural probe on these amnesic vectors
# _, _, proj_GPT_test_loss, proj_GPT_test_uuas = train((gold_distances_train, GPT_train_projected),
#                                                         (gold_distances_dev, GPT_dev_projected),
#                                                         (gold_distances_test, GPT_test_projected),
#                                                         device,
#                                                         epochs)
# print(f"Done training structural probe. This took {time.time() - t}s")

Training structural probe...


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0
torch.Size([24, 40, 40])
torch.Size([24, 40, 40])
torch.Size([24])
torch.Size([24, 51, 51])
torch.Size([24, 51, 51])
torch.Size([24])
torch.Size([24, 76, 76])
torch.Size([24, 76, 76])
torch.Size([24])
torch.Size([18, 60, 60])
torch.Size([18, 60, 60])
torch.Size([18])


  5%|▌         | 1/20 [00:00<00:11,  1.69it/s]

Epoch: 1
torch.Size([24, 40, 40])
torch.Size([24, 40, 40])
torch.Size([24])
torch.Size([24, 51, 51])
torch.Size([24, 51, 51])
torch.Size([24])
torch.Size([24, 76, 76])
torch.Size([24, 76, 76])
torch.Size([24])
torch.Size([18, 60, 60])
torch.Size([18, 60, 60])
torch.Size([18])


...

 15%|█▌        | 3/20 [00:01<00:09,  1.72it/s]

Epoch: 3
torch.Size([24, 40, 40])
torch.Size([24, 40, 40])
torch.Size([24])
torch.Size([24, 51, 51])
torch.Size([24, 51, 51])
torch.Size([24])
torch.Size([24, 76, 76])
torch.Size([24, 76, 76])
torch.Size([24])
torch.Size([18, 60, 60])
torch.Size([18, 60, 60])
torch.Size([18])


 20%|██        | 4/20 [00:02<00:10,  1.57it/s]

Epoch: 4
torch.Size([24, 40, 40])
torch.Size([24, 40, 40])
torch.Size([24])
torch.Size([24, 51, 51])
torch.Size([24, 51, 51])
torch.Size([24])


KeyboardInterrupt: 